# Solar Orbiter paper on first ICME April 2020


This is the source code for parts of Davies et al. (2021) A&A Solar Orbiter special issue.

Conda environment is "helio" as given by the files environment.yml and requirements.txt in this package.

### Figure 1, positions, in situ data Solar Orbiter (mag), Wind (mag and plasma)

In [3]:
import numpy as np
import scipy.io
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from matplotlib.dates import  DateFormatter
from datetime import timedelta
import seaborn as sns
import datetime
import astropy
import astropy.constants as constb
from sunpy.time import parse_time
import time
import pickle
import sys
import os
import urllib
import json
import importlib
import pandas as pd
import copy
import openpyxl
import h5py
import heliopy.data.spice as spicedata
import heliopy.spice as spice
import heliosat

#because script is not in root directory of package
#sys.path.append('/Users/chris/python/heliocats')
sys.path.append('/home/cmoestl/pycode/heliocats')
import heliocats.data as hd
from config import data_path

from astropy.constants import au
#define AU in km
AU=au.value/1e3

## Flux from 3DCORE for UMFR for OKane paper

Results for Davies and Forstner paper, Solar Orbiter fit, with D1AU and B1AU

$L_1$: 13.446 +/- 5.260

$L_2$: -4.870 +/- 4.732

$O$: 348.820 +/- 13.271

$D_{1AU}$: 0.114 +/- 0.022

$w$: 1.000 +/- 0.000

$\delta$: 2.000 +/- 0.000

$R_0$: 20.000 +/- 0.000

$V_0$: 307.068 +/- 42.834

$\tau$: -3.680 +/- 0.606

$n_a$: 1.140 +/- 0.000

$n_b$: 1.640 +/- 0.000

$B_{1AU}$: 14.278 +/- 0.873

$\gamma$: 1.213 +/- 0.490

$V_{SW}$: 296.659 +/- 30.239

$\sigma$: 1.500 +/- 0.000

In [6]:
#Wind: B0 (14.3 ± 0.9) nT, the model flux rope has a diameter of (0.114 ± 0.022) AU

#cylindrical toroidal, uniform twist Vandas & Romashets 2017
b0=14.3*1e-9 #nT to T
rt=0.114*AU*1e3 #AU to m

b0low=(14.3-0.9)*1e-9
rtlow=(0.114-0.022)*AU*1e3 #AU to m

b0up=(14.3+0.9)*1e-9
rtup=(0.114+0.022)*AU*1e3 #AU to m

a0=2.41
consts=2*np.pi/a0*scipy.special.j1(a0)

axflux=consts*(b0)*(rt**2)
axflux_low=consts*(b0low)*(rtlow**2)
axflux_up=consts*(b0up)*(rtup**2)

#b is twist, convert to per m instead of per AU
tau=3.7/(AU*1e3) #+/-0.6

tau_up=3.1/(AU*1e3) #+/-0.6
tau_low=4.3/(AU*1e3) #+/-0.6

axflux_uniform=(b0*np.pi/(tau**2))*np.log(1+(tau**2)*(rt**2))
axflux_uniform_low=(b0low*np.pi/(tau_low**2)*np.log(1+(tau_low**2)*(rtlow**2)))
axflux_uniform_up=(b0up*np.pi/(tau_up**2)*np.log(1+(tau_up**2)*(rtup**2)))

#1Wb is 10^8 Mx

print()
print('cylindrical, toroidal from Vandas MT solution, similar to lundquist')
print(np.round(axflux*1e-13,2), ' 10^21 Mx, 10^13 Wb')
print(np.round(axflux_low*1e-13,2),' to ',np.round(axflux_up*1e-13,2), '10^21 Mx, low, high')
print()
print()
print('cylindridal uniform twist formula')
print(np.round(axflux_uniform*1e-13,3), '10^21 Mx')
print(np.round(axflux_uniform_low*1e-13,2),' to ',np.round(axflux_uniform_up*1e-13,2), '10^21 Mx, low, high')


print()
print('numerical from 3DCORE apex with aspect ratio 2')
print( '???      ')


cylindrical, toroidal from Vandas MT solution, similar to lundquist
0.56  10^21 Mx, 10^13 Wb
0.34  to  0.85 10^21 Mx, low, high


cylindridal uniform twist formula
1.203 10^21 Mx
0.74  to  1.82 10^21 Mx, low, high

numerical from 3DCORE apex with aspect ratio 2
???      


## load in situ data, positions and THUX solar wind

In [ ]:
#the in situ data files are loaded from our server as a pickle

filemag=data_path+'solo_2020_icme_april_rtn.p'
sm=pickle.load(open(filemag, "rb" ) ) 

filewin=data_path+'wind_2018_2020_oct_gse.p'
[win,hwin]=pickle.load(open(filewin, "rb" ) )

filebepi='bepi_2020_icme_april_hee.p'
bm=pickle.load(open(data_path+filebepi, "rb" ) )      

#set bepi mag to nan after Apr 21
nanind=np.where(bm.time > parse_time('2020-04-21 18:00').datetime)

bm.bt[nanind]=np.nan
bm.bx[nanind]=np.nan
bm.by[nanind]=np.nan
bm.bz[nanind]=np.nan


#same for orbiter
nanind=np.where(np.logical_and(sm.time > parse_time('2020-04-21 00:00').datetime, sm.time < parse_time('2020-04-22 00:00').datetime)  )
sm.bt[nanind]=np.nan
sm.bx[nanind]=np.nan
sm.by[nanind]=np.nan
sm.bz[nanind]=np.nan

[psp, bepi, solo, sta, earth, venus, mars, mercury,frame]= \
      pickle.load( open( '../data/positions_psp_solo_bepi_sta_planets_HEEQ_1hour.p', "rb" ))


############################# HUX

wsa=np.loadtxt('../data/thux/wsa-vmap-19apr2020.txt')
wsa_all=np.loadtxt('../data/thux/wsa-planets-missions-cr2047-cr2234.txt',skiprows=1)

#convert matlab time to matplotlib time

wsa_all_datetime=[None]*len(wsa_all)
for p in np.arange(len(wsa_all)):
        wsa_all_datetime[p]= datetime.datetime.fromordinal(wsa_all[p,0].astype(int) ) + \
        datetime.timedelta(days=wsa_all[p,0]%1) - datetime.timedelta(days = 366) 


#vEarth	vMercury	vVenus	vMars	vBepi	vPSP	vSOLO	vSTEREOA	vSTEREOB
wsa_all_earth=wsa_all[:,1]
wsa_all_solo=wsa_all[:,7]
wsa_all_sta=wsa_all[:,8]



################### THUX

wsa_thux=np.loadtxt('../data/thux/wsa-vmap-19apr2020-v2.txt')
wsa_apr_thux=np.loadtxt('../data/thux/wsa-planets-missions-19apr2020.txt',skiprows=1)

#convert matlab time to matplotlib time

wsa_apr_thux_datetime=[None]*len(wsa_apr_thux)
for p in np.arange(len(wsa_apr_thux)):
        wsa_apr_thux_datetime[p]= datetime.datetime.fromordinal(wsa_apr_thux[p,0].astype(int) ) + \
        datetime.timedelta(days=wsa_apr_thux[p,0]%1) - datetime.timedelta(days = 366) 


#vEarth	vMercury	vVenus	vMars	vBepi	vPSP	vSOLO	vSTEREOA	vSTEREOB
wsa_apr_thux_earth=wsa_apr_thux[:,1]
wsa_apr_thux_solo=wsa_apr_thux[:,7]
wsa_apr_thux_sta=wsa_apr_thux[:,8]


print('done')

In [ ]:
def plot_bgsw(ax,cbarax):

    
    #for rotation
    #k=0
    #CR 2229	2020 Mar 28	0853 start
    #k=14
    #rotSun = 27.2753
    #rotAngle = (2 * np.pi / rotSun) * k #+np.pi/2 
    #rotAngle = (2 * np.pi / rotSun) * k  

    thetaLen=180
    rLen=425
    r_sun = 695700.
    au = 149597870.
    startBGSW = 5

    #grid
    angle = np.deg2rad(np.arange(0, 362, 362 / thetaLen)) #+ np.deg2rad(-90) #+ rotAngle    
    radius = np.arange(startBGSW, rLen + startBGSW) / au * r_sun
    thetaBGSW, rBGSW = np.meshgrid(angle, radius)

    ## HUX
    
    #wsafin =copy.deepcopy(wsa)
    
    #levels = np.arange(np.round(np.min(wsafin))+1, np.max(wsafin), 5)

    #cf = ax.contourf(thetaBGSW.T, rBGSW.T, np.flip(wsafin,axis=1).T, levels,
    #                 cmap=plt.cm.get_cmap('coolwarm'), alpha=0.9,
    #                 vmin=np.min(wsafin), vmax=np.max(wsafin),antialiased=True)

    #cf = ax.contourf(thetaBGSW.T, rBGSW.T, wsafin.T,levels,
    #                 cmap=plt.cm.get_cmap('coolwarm'), alpha=0.5,antialiased=True)

    #cf = ax.contourf(thetaBGSW.T, rBGSW.T, np.flip(wsafin,axis=1).T,levels,
    #                 cmap=plt.cm.get_cmap('coolwarm'), alpha=0.5,antialiased=True)

    ##### THUX

    levels = np.arange(np.round(np.min(wsa_thux)), np.max(wsa_thux), 5)
    
    cf = ax.contourf(thetaBGSW.T, rBGSW.T, np.flip(wsa_thux,axis=1).T,levels,
                     cmap=plt.cm.get_cmap('coolwarm'),vmin=np.min(wsa_thux), 
                     vmax=np.max(wsa_thux),alpha=0.6,antialiased=True)


    
    # This is the fix for the white lines between contour levels
    #for c in cf.collections:
    #    c.set_edgecolor('face')
    #    c.set_linewidth( 0.1 ) 
        
        

    cax = plt.axes(cbarax)
    cbar = plt.colorbar(cf, cax=cax, ticks=np.arange(200, 600, 50),orientation="horizontal")
    ticklabs = cbar.ax.get_yticklabels()
    cbar.ax.set_yticklabels(ticklabs, fontsize=15)
    cbar.set_label('Solar wind speed [km/s]', fontsize=15)


    #ax.set_ylim(0, 1.2) 
    #ax.grid(False)


plt.figure(1,figsize=(10, 10),dpi=50)


ax1 = plt.subplot2grid((1, 1), (0, 0),projection='polar') 


cbarax1=[0.2, 0.05, 0.6, 0.02]

plot_bgsw(ax1,cbarax1)
#plot_bgsw(ax1,cbarax1)


## Results

In [ ]:
#%matplotlib inline
#%matplotlib
#hp.plot_insitu_measure(sm, '2020-Apr-18 12:00','2020-Apr-21 12:00', 'Solo', 'results/')
#hp.plot_insitu_measure(win, '2020-Apr-18 12:00','2020-Apr-23 12:00', 'Wind', 'results/')


######################### solo
sicme_start_time=parse_time('2020-04-19T05:06Z').datetime
smo_start_time=parse_time('2020-04-19T08:59Z').datetime
smo_end_time=parse_time('2020-04-20T09:15Z').datetime


#unperturbed Solar Orbiter: 2020-04-19 13:56:30 to 2020-04-20 04:36:30
smo_start_un=parse_time('2020-04-19T13:56Z').datetime
smo_end_un=parse_time('2020-04-20T04:36Z').datetime


print('SolO times:',sicme_start_time, smo_start_time,smo_end_time,smo_start_un,smo_end_un)


print()
#index of times
sicme_start_ind=np.where(sm.time>sicme_start_time)[0][0]
smo_start_ind=np.where(sm.time>smo_start_time)[0][0]
smo_end_ind=np.where(sm.time>smo_end_time)[0][0]

smo_start_un_ind=np.where(sm.time>smo_start_un)[0][0]
smo_end_un_ind=np.where(sm.time>smo_end_un)[0][0]



print('HEEQ position of SolO r/lat/lon at icme_start_time:',np.round(sm.r[sicme_start_ind],3),np.round(sm.lat[sicme_start_ind],2),np.round(sm.lon[sicme_start_ind],2))
print('HEEQ position of SolO r/lat/lon at mo_start_time:',np.round(sm.r[smo_start_ind],3),np.round(sm.lat[smo_start_ind],2),np.round(sm.lon[smo_start_ind],2))
print('HEEQ position of SolO r/lat/lon at mo_end_time:',np.round(sm.r[smo_end_ind],3),np.round(sm.lat[smo_end_ind],2),np.round(sm.lon[smo_end_ind],2))


print('SolO ICME duration hours ', np.round((smo_end_time-sicme_start_time).total_seconds()/3600,2))
print('SolO sheath duration hours ', np.round((smo_start_time-sicme_start_time).total_seconds()/3600,2))
print('SolO MO duration hours ', np.round((smo_end_time-smo_start_time).total_seconds()/3600,2))
print('SolO MO unperturbed hours ', np.round((smo_end_un-smo_start_un).total_seconds()/3600,2))
print()

print('SOLO Bmean ICME   ',np.round(np.nanmean(sm.bt[sicme_start_ind:smo_end_ind]),1))
print('SOLO Bmean sheath ',np.round(np.nanmean(sm.bt[sicme_start_ind:smo_start_ind]),1))
print('SOLO Bmean MO     ',np.round(np.nanmean(sm.bt[smo_start_ind:smo_end_ind]),1))
print('SOLO Bmean UMFR     ',np.round(np.nanmean(sm.bt[smo_start_un_ind:smo_end_un_ind]),1))
print()


print()

#arrival speed für solo laut elevohi ist 377+/-8 km/s
#und für earth 395+/-15 km/s
#bei Erde in situ 346.3 km/s

print('Solo MO unperturbed, radial size s=v*t sum data points with 1 minute resolution with hi speed 377-49 km/s ')

#use unperturbed interval
smo_radialsize_un=np.round((377-49)*(smo_end_un_ind-smo_start_un_ind)*60/AU,4)
print(smo_radialsize_un, 'AU')




print('------------------------------------------')



########################## Wind
wicme_start_time=parse_time('2020-04-20T01:34Z').datetime
wmo_start_time=parse_time('2020-04-20T07:56Z').datetime
#alternative wmo_start_time=parse_time('2020-04-20T06:37Z').datetime
wmo_end_time=parse_time('2020-04-21T11:18Z').datetime


#unperturbed: Wind: 2020-04-20 11:27:30 to 2020-04-21 02:09:30
wmo_start_un=parse_time('2020-04-20T11:28Z').datetime
#alternative wmo_start_time=parse_time('2020-04-20T06:37Z').datetime
wmo_end_un=parse_time('2020-04-21T02:10Z').datetime




#index of times
wicme_start_ind=np.where(wicme_start_time==win.time)[0][0]
wmo_start_ind=np.where(wmo_start_time==win.time)[0][0]
wmo_end_ind=np.where(wmo_end_time==win.time)[0][0]
wmo_start_un_ind=np.where(wmo_start_un==win.time)[0][0]
wmo_end_un_ind=np.where(wmo_end_un==win.time)[0][0]



print()
print('Wind times:',wicme_start_time, wmo_start_time,wmo_end_time,wmo_start_un,wmo_end_un)
print()
print('from data file')
print('HEEQ position of Wind r/lat/lon at icme_start_time:',np.round(win.r[wicme_start_ind],3),np.round(win.lat[wicme_start_ind],2),np.round(win.lon[wicme_start_ind],2))
print('HEEQ position of Wind r/lat/lon at mo_start_time:',np.round(win.r[wmo_start_ind],3),np.round(win.lat[wmo_start_ind],2),np.round(win.lon[wmo_start_ind],2))
print('HEEQ position of Wind r/lat/lon at mo_end_time:',np.round(win.r[wmo_end_ind],3),np.round(win.lat[wmo_end_ind],2),np.round(win.lon[wmo_end_ind],2))
print()

print('with heliosat')
#t_snap_wind = datetime.datetime(2020, 4, 20, 1, 34)
#wind_pos = heliosat.WIND().trajectory(t_snap_wind, frame="HEEQ")
#print("wind lon: ", np.arctan2(wind_pos[1], wind_pos[0]) * 360 / 2 / np.pi)
#print("wind lat: ", np.arctan2(wind_pos[2], wind_pos[0]) * 360 / 2 / np.pi)

wind1=heliosat.WIND().trajectory(wicme_start_time, frame="HEEQ")
[wind1r, wind1lat, wind1lon]=hd.cart2sphere(wind1[0],wind1[1],wind1[2])
wind1lon=np.rad2deg(wind1lon) #convert to degree
wind1lat=np.rad2deg(wind1lat)

wind2=heliosat.WIND().trajectory(wmo_start_time, frame="HEEQ")
[wind2r, wind2lat, wind2lon]=hd.cart2sphere(wind2[0],wind2[1],wind2[2])
wind2lon=np.rad2deg(wind2lon) #convert to degree
wind2lat=np.rad2deg(wind2lat)

wind3=heliosat.WIND().trajectory(wmo_end_time, frame="HEEQ")
[wind3r, wind3lat, wind3lon]=hd.cart2sphere(wind3[0],wind3[1],wind3[2])
wind3lon=np.rad2deg(wind3lon) #convert to degree
wind3lat=np.rad2deg(wind3lat)


print('HEEQ position of Wind r/lat/lon at icme_start_time:',np.round(wind1r,3),np.round(wind1lat,2),np.round(wind1lon,2))
print('HEEQ position of Wind r/lat/lon at mo_start_time:',np.round(wind2r,3),np.round(wind2lat,2),np.round(wind2lon,2))
print('HEEQ position of Wind r/lat/lon at mo_end_time:',np.round(wind3r,3),np.round(wind3lat,2),np.round(wind3lon,2))

print()


print('Wind ICME duration hours ', np.round((wmo_end_time-wicme_start_time).total_seconds()/3600,2))
print('Wind sheath duration hours ', np.round((wmo_start_time-wicme_start_time).total_seconds()/3600,2))
wmo_dur=np.round((wmo_end_time-wmo_start_time).total_seconds()/3600,2)
print('Wind MO duration hours ', wmo_dur)
print('Wind MO unperturbed hours ', np.round((wmo_end_un-wmo_start_un).total_seconds()/3600,2))
print('Wind MO unperturbed hours ', (wmo_end_un-wmo_start_un).total_seconds()/3600)


print()
print('Wind Bmean ICME   ',np.round(np.nanmean(win.bt[wicme_start_ind:wmo_end_ind]),1))
print('Wind Bmean sheath ',np.round(np.nanmean(win.bt[wicme_start_ind:wmo_start_ind]),1))
print('Wind Bmean MO     ',np.round(np.nanmean(win.bt[wmo_start_ind:wmo_end_ind]),1))
print('Wind Bmean UMFR   ',np.round(np.nanmean(win.bt[wmo_start_un_ind:wmo_end_un_ind]),1))
print()
print('Wind Vmean ICME   ',np.round(np.nanmean(win.vt[wicme_start_ind:wmo_end_ind]),1))


print()
print('Wind MO unperturbed, radial size s=v*t for each timestep, sum, wind 2 minute resolution: ')
#use unperturbed interval
wmo_radialsize_un=np.round(np.sum(win.vt[wmo_start_un_ind:wmo_end_un_ind]*120/AU),5)
print(wmo_radialsize_un, 'AU')

print('------------------------------------------')





########################## Bepi
bicme_start_time=parse_time('2020-04-20T03:09Z').datetime
bmo_start_time=parse_time('2020-04-20T08:05Z').datetime
bmo_end_time=parse_time('2020-04-21T10:08Z').datetime


#unperturbed Bepi: 2020-04-20 14:03:00 to 2020-04-21 03:18:00

bmo_start_un=parse_time('2020-04-20T14:03Z').datetime
#alternative wmo_start_time=parse_time('2020-04-20T06:37Z').datetime
bmo_end_un=parse_time('2020-04-21T03:18Z').datetime


#index of times
bicme_start_ind=np.where(bicme_start_time==bm.time)[0][0]
bmo_start_ind=np.where(bmo_start_time==bm.time)[0][0]
bmo_end_ind=np.where(bmo_end_time==bm.time)[0][0]

bmo_start_un_ind=np.where(bmo_start_un==bm.time)[0][0]
bmo_end_un_ind=np.where(bmo_end_un==bm.time)[0][0]



print('Bepi times:',bicme_start_time, bmo_start_time,bmo_end_time,bmo_start_un,bmo_end_un)
print()

print('HEEQ position of Bepi r/lat/lon at icme_start_time:',np.round(bm.r[bicme_start_ind],3),np.round(bm.lat[bicme_start_ind],2),np.round(bm.lon[bicme_start_ind],2))
print('HEEQ position of Bepi r/lat/lon at mo_start_time:',np.round(bm.r[bmo_start_ind],3),np.round(bm.lat[bmo_start_ind],2),np.round(bm.lon[bmo_start_ind],2))
print('HEEQ position of Bepi r/lat/lon at mo_end_time:',np.round(bm.r[bmo_end_ind],3),np.round(bm.lat[bmo_end_ind],2),np.round(bm.lon[bmo_end_ind],2))
print()


print('with heliosat')
#t_snap_wind = datetime.datetime(2020, 4, 20, 1, 34)
#wind_pos = heliosat.WIND().trajectory(t_snap_wind, frame="HEEQ")
#print("wind lon: ", np.arctan2(wind_pos[1], wind_pos[0]) * 360 / 2 / np.pi)
#print("wind lat: ", np.arctan2(wind_pos[2], wind_pos[0]) * 360 / 2 / np.pi)

b1=heliosat.BEPI().trajectory(bicme_start_time, frame="HEEQ")
[b1r, b1lat, b1lon]=hd.cart2sphere(b1[0],b1[1],b1[2])
b1lon=np.rad2deg(b1lon) #convert to degree
b1lat=np.rad2deg(b1lat)

b2=heliosat.BEPI().trajectory(bmo_start_time, frame="HEEQ")
[b2r, b2lat, b2lon]=hd.cart2sphere(b2[0],b2[1],b2[2])
b2lon=np.rad2deg(b2lon) #convert to degree
b2lat=np.rad2deg(b2lat)

b3=heliosat.BEPI().trajectory(bmo_end_time, frame="HEEQ")
[b3r, b3lat, b3lon]=hd.cart2sphere(b3[0],b3[1],b3[2])
b3lon=np.rad2deg(b3lon) #convert to degree
b3lat=np.rad2deg(b3lat)


print('HEEQ position of Bepi r/lat/lon at icme_start_time:',np.round(b1r,3),np.round(b1lat,2),np.round(b1lon,2))
print('HEEQ position of Bepi r/lat/lon at mo_start_time:',np.round(b2r,3),np.round(b2lat,2),np.round(b2lon,2))
print('HEEQ position of Bepi r/lat/lon at mo_end_time:',np.round(b3r,3),np.round(b3lat,2),np.round(b3lon,2))

print()


print('Bepi ICME duration hours ', np.round((bmo_end_time-bicme_start_time).total_seconds()/3600,2))
print('Bepi sheath duration hours ', np.round((bmo_start_time-bicme_start_time).total_seconds()/3600,2))
print('Bepi MO duration hours ', np.round((bmo_end_time-bmo_start_time).total_seconds()/3600,2))
print('Bepi MO unperturbed hours ', np.round((bmo_end_un-bmo_start_un).total_seconds()/3600,2))
print()
print('Bepi Bmean ICME   ',np.round(np.nanmean(bm.bt[bicme_start_ind:bmo_end_ind]),1))
print('Bepi Bmean sheath ',np.round(np.nanmean(bm.bt[bicme_start_ind:bmo_start_ind]),1))
print('Bepi Bmean MO     ',np.round(np.nanmean(bm.bt[bmo_start_ind:bmo_end_ind]),1))
print('Bepi Bmean UMFR   ',np.round(np.nanmean(bm.bt[bmo_start_un_ind:bmo_end_un_ind]),1))
print()
print('----------------------------')
print()

print('radial separation SolO Wind ',0.996-0.802)
print('radial separation SolO Bepi ',0.996-1.011)

## Flux rope aspect ratio plot, derived from STEREO-A HI observations

In [ ]:
print('Flux rope aspect ratio')
print()
print('from HI measurements')

########### get apex distance
#wie is da R berechnet?
#ich hab die parameter mit abgespeichert
#phi = 82°, half width = 35°, f = 0.7
#es gibt insgesamt 4 runs, die genau treffen. einer davon hat ausreißerparameter, aber die anderen drei sind sich ziemlich einig.
#stereo - earth war -75°, also ist das eine direction von 7° from earth

from scipy.io.idl import readsav
aw_data_r_file='../data/davies_2020_paper/angular_width_r.sav'
aw_data_r=readsav(aw_data_r_file)
aw_r_apex=aw_data_r['r_apex']


############ get x and y data
ar_data_file='../data/davies_2020_paper/20200414_aspectratio_long.txt'
ar_data=np.genfromtxt(ar_data_file)
#get time ...

ar_time=np.arange(len(ar_data))[1:]

#TRACK_DATE , 1 y_top, 2 y_bottom, 3 y_left, 4 y_right, 5 y_center, 6 x_top, 7 x_bottom, 8 x_left,9 x_right, 10 x_center
y_top=ar_data[:,1][1:]
y_bottom=ar_data[:,2][1:]
yex=y_top-y_bottom
yex_mean=np.mean(yex)
yex_std=np.std(yex)

x_left=ar_data[:,8][1:]
x_right=ar_data[:,9][1:]
xex=x_right-x_left

xex_mean=(np.nanmean(xex))
xex_std=(np.nanstd(xex))

x_center=ar_data[:,10][1:]


print('y')
print(yex_mean)
print(yex_std)

print()
print('x')
print(xex_mean)
print(xex_std)


print()

#https://en.wikipedia.org/wiki/Propagation_of_uncertainty#Non-linear_combinations
covxy = np.cov(xex, yex, bias=True)[0][1]
aspect_direct=yex/xex
#boundaries with f=A/B error propagation
aspect_direct_p=yex/xex+np.sqrt((xex_std/xex)**2+  (yex_std/yex)**2+ 2*(yex_std/yex)**2*covxy/ (yex*xex) )
aspect_direct_m=yex/xex-np.sqrt((xex_std/xex)**2+  (yex_std/yex)**2+ 2*(yex_std/yex)**2*covxy/ (yex*xex) )

#plt.plot(aw_r_apex,yex_mean/xex_mean)

print('aspect ratio measurements')

print(aspect_direct)
print(aspect_direct_p)
print(aspect_direct_m)


aspect_direct_mean=np.round(np.mean(aspect_direct),2)
aspect_direct_std=np.round(np.std(aspect_direct),2)

print()
print('-------------- void aspect ratio directly from images')
print('mean of aspect ratio',aspect_direct_mean,' +/- standard ',aspect_direct_std)


print()
print('------------- void angular width')

#Tanja: angular_width = (atan(((y_top+abs(y_bottom))/2.)/x_center)/!dtor)*2

aw_calc=np.arctan(  (  yex*0.5 ) / x_center ) *2*180/np.pi
print(aw_calc)

aw_calc_mean=np.round(np.mean(aw_calc),2)
aw_calc_std=np.round(np.std(aw_calc),2)

print()
print('mean of angular width',aw_calc_mean,' +/- standard ',aw_calc_std)
#plt.plot(aw_r_apex,aw_calc)

## Figure aspect ratio

In [ ]:
sns.set_context("talk")     
#sns.set_style('darkgrid')

sns.set_style('whitegrid')
#sns.set_style("ticks",{'grid.linestyle': '--'})
plt.figure(10,figsize=(10, 10),dpi=100)

#define subplot grid
ax1 = plt.subplot2grid((2, 1), (1, 0))  #solo b
ax2 = plt.subplot2grid((2, 1), (0, 0))  #bepib

ax2.plot(np.arange(np.min(aw_r_apex),np.max(aw_r_apex),0.001),np.zeros(140)+aspect_direct_mean,color='orangered',label='mean $\delta$')
ax2.plot(np.arange(np.min(aw_r_apex),np.max(aw_r_apex),0.001),np.zeros(140)+aspect_direct_mean+aspect_direct_std,color='orangered',linestyle='--',label='mean $\delta$ +/- std $\delta$')
ax2.plot(np.arange(np.min(aw_r_apex),np.max(aw_r_apex),0.001),np.zeros(140)+aspect_direct_mean-aspect_direct_std,color='orangered',linestyle='--')
ax2.plot(np.arange(0,1,0.1),np.zeros(10)+1,color='mediumslateblue',linestyle='-',lw=2,label='circular cross section $\delta=1$')
ax2.plot(aw_r_apex,yex/xex,'ok',markersize=5,label='aspect ratio $\delta$ of void in STEREO-A HI1')

ax2.legend(loc=2,fontsize=14)
ax2.set_ylabel('aspect ratio $\delta$')
ax2.set_ylim(0.5,3.3)
ax2.set_xlim(np.min(aw_r_apex)-0.01,np.max(aw_r_apex)+0.01)


ax1.plot(np.arange(np.min(aw_r_apex),np.max(aw_r_apex),0.001),np.zeros(140)+np.mean(aw_calc),color='orangered',label='mean $a_w$')
ax1.plot(np.arange(np.min(aw_r_apex),np.max(aw_r_apex),0.001),np.zeros(140)+np.mean(aw_calc)+np.std(aw_calc),color='orangered',linestyle='--',label='mean $a_w$ +/- std $a_w$')
ax1.plot(np.arange(np.min(aw_r_apex),np.max(aw_r_apex),0.001),np.zeros(140)+np.mean(aw_calc)-np.std(aw_calc),color='orangered',linestyle='--')
ax1.plot(aw_r_apex,aw_calc,'ok',markersize=5,label='$a_w$ of void in STEREO-A HI1')

ax1.legend(loc=2,fontsize=14)
ax1.set_ylabel('angular width $a_w$ [°]')
ax1.set_ylim(10,20)
ax1.set_xlim(np.min(aw_r_apex)-0.01,np.max(aw_r_apex)+0.01)
ax1.set_xlabel('Heliocentric distance [AU]')




plt.tight_layout()

plt.annotate('(a)',[0.005,0.96],xycoords='figure fraction',fontsize=20)
plt.annotate('(b)',[0.005,0.48],xycoords='figure fraction',fontsize=20)

plt.savefig('/home/cmoestl/pycode/heliocats/results/davies_2020/figure_aspect_aw.png',dpi=300)
plt.savefig('/home/cmoestl/pycode/heliocats/results/davies_2020/figure_aspect_aw.pdf')

### Estimation of aspect ratio at 1 AU

In [ ]:
print()
print('Aspect ratio at 1 AU from HI angular width measurements')
print()
from scipy.io.idl import readsav
aw_data_file='../data/davies_2020_paper/angular_width.sav'
aw_data=readsav(aw_data_file)


aw_mean=np.round(np.mean(aw_data['angular_width']),2)
aw_std=np.round(np.std(aw_data['angular_width']),2)

print('mean aw',aw_mean, ' +/- ',aw_std )
print('(constant angular width = self similar expansion)')

############################### with 1 AU circle
d_lat=np.round(2*np.pi/(360/aw_mean),3)
d_lat_p=np.round(2*np.pi/(360/(aw_mean+aw_std)),3)
d_lat_m=np.round(2*np.pi/(360/(aw_mean-aw_std)),3)


print('d_latitude',d_lat, 'AU, range:',d_lat_m,' - ',d_lat_p,'AU')
print('d_unperturbed',wmo_radialsize_un,'AU')

aspect_ratio_void=np.round(d_lat/wmo_radialsize_un,2)
aspect_ratio_void_p=np.round(d_lat_p/wmo_radialsize_un,2)
aspect_ratio_void_m=np.round(d_lat_m/wmo_radialsize_un,2)

print('aspect_ratio with 1 AU circle',aspect_ratio_void, 'range:',aspect_ratio_void_p,aspect_ratio_void_m)
print()

############################### with straight vertical line at 1 AU

d_lat2  =np.round(  2*np.tan(aw_mean/2*np.pi/180)   ,3)
d_lat2_p=np.round(  2*np.tan((aw_mean+aw_std)/2*np.pi/180)   ,3)
d_lat2_m=np.round(  2*np.tan((aw_mean-aw_std)/2*np.pi/180)   ,3)

print('d_latitude2',d_lat2, 'AU, range:',d_lat2_m,' - ',d_lat2_p,'AU')
print('d_unperturbed',wmo_radialsize_un,'AU')

aspect_ratio_void2  =np.round(d_lat2 /wmo_radialsize_un,2)
aspect_ratio_void2_p=np.round(d_lat2_p/wmo_radialsize_un,2)
aspect_ratio_void2_m=np.round(d_lat2_m/wmo_radialsize_un,2)

print('aspect_ratio with 1 AU straight vertical line',aspect_ratio_void2, 'range:',aspect_ratio_void2_p,aspect_ratio_void2_m)
print()



print()
print()

aw_full=45
print('full extent degree from HELCATS catalog', aw_full)
d_lat_full=np.round(2*np.pi/(360/aw_full),2)
print('d_latitude_full',d_lat_full)

aspect_ratio_full=aspect_ratio_void=np.round(d_lat_full/wmo_radialsize_un,2)
print('aspect_ratio full',aspect_ratio_full)

### Make Figure 1 plot 

In [ ]:
sns.set_context("talk")     
#sns.set_style('darkgrid')
sns.set_style('whitegrid')
sns.set_style("ticks",{'grid.linestyle': '--'})


#set wind awkward data points to nan
windspike_ind=np.where(win.time > parse_time('2020-04-20 17:08').datetime)[0]
win.bt[windspike_ind[0:2]]=np.nan
win.bx[windspike_ind[0:2]]=np.nan
win.by[windspike_ind[0:2]]=np.nan
win.bz[windspike_ind[0:2]]=np.nan


#set wind awkward data points to nan
windspike_ind=np.where(win.time > parse_time('2020-04-21 01:20').datetime)[0]
win.bt[windspike_ind[0:1]]=np.nan
win.bx[windspike_ind[0:1]]=np.nan
win.by[windspike_ind[0:1]]=np.nan
win.bz[windspike_ind[0:1]]=np.nan


plotstart=datetime.datetime(2020, 4, 18)
plotend=datetime.datetime(2020, 4,23)
blim=25





plt.figure(1,figsize=(20, 13),dpi=100)

#define subplot grid
ax1 = plt.subplot2grid((4, 2), (0, 1))  #solo b
ax2 = plt.subplot2grid((4, 2), (1, 1))  #bepib
ax3 = plt.subplot2grid((4, 2), (2, 1))  #wind b
ax4 = plt.subplot2grid((4, 2), (3, 1))  #wind v


ax5 = plt.subplot2grid((4, 2), (0, 0), rowspan=4, projection='polar') #positions


#########################################################################
backcolor='black'
psp_color='black'
bepi_color='blue'
solo_color='coral'

symsize_planet=200
symsize_spacecraft=150
fsize=20
plot_orbit=True
fadeind=150

plot_parker=False
plot_hux=True


AUkm=149597870.7   

#for parker spiral   
theta=np.arange(0,np.deg2rad(180),0.01)

#Coordinate System
#frame='HCI'
frame='HEEQ'
print(frame)

#sidereal solar rotation rate
if frame=='HCI': sun_rot=24.47
#synodic
if frame=='HEEQ': sun_rot=26.24

frame_time_num=mdates.date2num(parse_time('2020-Apr-20').datetime)
print( 'current frame_time_num', str(mdates.num2date(frame_time_num)))

#these have their own times
dct=frame_time_num-psp.time
psp_timeind=np.argmin(abs(dct))

dct=frame_time_num-bepi.time
bepi_timeind=np.argmin(abs(dct))

dct=frame_time_num-solo.time
solo_timeind=np.argmin(abs(dct))

dct=frame_time_num-sta.time
sta_timeind=np.argmin(abs(dct))

#all same times
dct=frame_time_num-earth.time
earth_timeind=np.argmin(abs(dct))


ax5.scatter(venus.lon[earth_timeind], venus.r[earth_timeind]*np.cos(venus.lat[earth_timeind]), s=symsize_planet, c='orange', alpha=1,lw=0,zorder=3)
ax5.scatter(mercury.lon[earth_timeind], mercury.r[earth_timeind]*np.cos(mercury.lat[earth_timeind]), s=symsize_planet, c='dimgrey', alpha=1,lw=0,zorder=3)
ax5.scatter(earth.lon[earth_timeind], earth.r[earth_timeind]*np.cos(earth.lat[earth_timeind]), s=symsize_planet, c='mediumseagreen', alpha=1,lw=0,zorder=3)
ax5.scatter(sta.lon[earth_timeind], sta.r[earth_timeind]*np.cos(sta.lat[earth_timeind]), s=symsize_spacecraft, c='red', marker='s', alpha=1,lw=0,zorder=3)
ax5.scatter(mars.lon[earth_timeind], mars.r[earth_timeind]*np.cos(mars.lat[earth_timeind]), s=symsize_planet, c='orangered', alpha=1,lw=0,zorder=3)

ax5.scatter(psp.lon[psp_timeind], psp.r[psp_timeind]*np.cos(psp.lat[psp_timeind]), s=symsize_spacecraft, c=psp_color, marker='s', alpha=1,lw=0,zorder=3)
ax5.scatter(bepi.lon[bepi_timeind], bepi.r[bepi_timeind]*np.cos(bepi.lat[bepi_timeind]), s=symsize_spacecraft, c=bepi_color, marker='s', alpha=1,lw=0,zorder=3)
ax5.scatter(solo.lon[solo_timeind], solo.r[solo_timeind]*np.cos(solo.lat[solo_timeind]), s=symsize_spacecraft, c=solo_color, marker='s', alpha=1,lw=0,zorder=3)

#set axes and grid
ax5.set_theta_zero_location('E')
#plt.thetagrids(range(0,360,45),(u'0\u00b0 '+frame+' longitude',u'45\u00b0',u'90\u00b0',u'135\u00b0',u'+/- 180\u00b0',u'- 135\u00b0',u'- 90\u00b0',u'- 45\u00b0'), ha='right', fmt='%d',fontsize=fsize-1,color=backcolor, alpha=0.9)
plt.thetagrids(range(0,360,45),(u'0\u00b0',u'45\u00b0',u'90\u00b0',u'135\u00b0',u'+/-180\u00b0',u'-135\u00b0',u'-90\u00b0',u'-45\u00b0'), fmt='%d',fontsize=fsize+1,color=backcolor, alpha=0.9,zorder=4)

#plt.rgrids((0.10,0.39,0.72,1.00,1.52),('0.10','0.39','0.72','1.0','1.52 AU'),angle=125, fontsize=fsize,alpha=0.9, color=backcolor)
plt.rgrids((0.2,0.4,0.6,0.8,1.0),('0.2','0.4','0.6','0.8','1.0 AU'),angle=75, fontsize=fsize+1,alpha=0.9, color=backcolor,zorder=4)

#ax.set_ylim(0, 1.75) #with Mars
ax5.set_ylim(0, 1.2) 

#Sun
ax5.scatter(0,0,s=100,c='yellow',alpha=1, edgecolors='black', linewidth=0.3)


#spacecraft labels
ax5.text(psp.lon[psp_timeind]-0.2,psp.r[psp_timeind],'Parker Solar Probe', color=psp_color, ha='center',fontsize=fsize-4,verticalalignment='top')
ax5.text(solo.lon[solo_timeind]-0.05,solo.r[solo_timeind],'Solar Orbiter', color=solo_color, ha='right',fontsize=fsize-4,verticalalignment='top')
ax5.text(bepi.lon[bepi_timeind]-0.06,bepi.r[bepi_timeind],'Bepi', color=bepi_color, ha='center',fontsize=fsize-4,verticalalignment='top')
ax5.text(bepi.lon[bepi_timeind]-0.16,bepi.r[bepi_timeind],'Colombo', color=bepi_color, ha='center',fontsize=fsize-4,verticalalignment='top')
ax5.text(sta.lon[sta_timeind]-0.16,sta.r[sta_timeind]+0.01,'STEREO-A', color='red', ha='center',fontsize=fsize-4,verticalalignment='top')
ax5.text(earth.lon[earth_timeind]+0.12,earth.r[earth_timeind],'Earth', color='green', ha='center',fontsize=fsize-4,verticalalignment='top')


if plot_hux:
    cbarax=[0.08,0.13,0.34,0.02]
    plot_bgsw(ax5,cbarax)
   
    

#parker spiral
if plot_parker:
    for q in np.arange(0,12):
        omega=2*np.pi/(sun_rot*60*60*24) #solar rotation in seconds
        v=400/AUkm #km/s
        r0=695000/AUkm
        r=v/omega*theta+r0*7
        ax5.plot(-theta+np.deg2rad(0+(360/24.47)*1+360/12*q), r, alpha=0.4, lw=0.8,color='black',zorder=1)



#---------------------------------------- Solo MAG

ax1.plot(sm.time, sm.bt, "k-", lw=1,label='Btotal')
ax1.plot(sm.time, sm.bx, "r-", color="xkcd:magenta", lw=1, label='Br')
ax1.plot(sm.time, sm.by, "g-", color="xkcd:orange", lw=1,label='Bt')
ax1.plot(sm.time, sm.bz, "b-", color="xkcd:azure", lw=1,label='Bn')
    
#plot vertical lines
ax1.plot_date([sicme_start_time,sicme_start_time],[-500,500],'-k',linewidth=1)            
ax1.plot_date([smo_start_time,smo_start_time],[-500,500],'-k',linewidth=1)            
ax1.plot_date([smo_end_time,smo_end_time],[-500,500],'-k',linewidth=1)            

ax1.set_ylabel('B [nT, RTN]')
ax1.set_xlim(plotstart,plotend)
ax1.set_ylim(-blim,blim)
ax1.legend(loc=1,ncol=4,fontsize=11)
ax1.grid(True)
ax1.xaxis.set_minor_locator(matplotlib.ticker.MultipleLocator(1/24*6))

#position on figure
#textstart=plotstart+datetime.timedelta(hours=2)
#plt.text(textstart,25,'HEEQ R/lat/lon: '+str(np.round(sm.r[sicme_start_ind],3))+' ' +str(np.round(sm.lat[sicme_start_ind],2))+'  '+\
#         str(np.round(sm.lon[sicme_start_ind],2)),fontsize=13)



#--------------------------------------- Bepi MPO MAG

ax2.plot(bm.time, bm.bt, "k-", lw=1,label='Btotal')
ax2.plot(bm.time, bm.bx,'-r', color="xkcd:magenta",linewidth=1,label='Bx')
ax2.plot(bm.time, bm.by, "g-", color="xkcd:orange", lw=1,label='By')
ax2.plot(bm.time, bm.bz, "b-", color="xkcd:azure", lw=1,label='Bz')

    
#plot vertical lines
ax2.plot_date([bicme_start_time,bicme_start_time],[-500,500],'-k',linewidth=1)            
ax2.plot_date([bmo_start_time,bmo_start_time],[-500,500],'-k',linewidth=1)            
ax2.plot_date([bmo_end_time,bmo_end_time],[-500,500],'-k',linewidth=1)            

ax2.set_ylabel('B [nT, HEE]')
ax2.set_xlim(plotstart,plotend)
ax2.set_ylim(-blim,blim)
ax2.legend(loc=1,ncol=1,fontsize=11)
ax2.grid(True)
ax2.xaxis.set_minor_locator(matplotlib.ticker.MultipleLocator(1/24*6))




############################## Wind MFI
    
#plt.title('Wind MFI')

ax3.plot(win.time, win.bt, "k-", lw=1,label='Btotal')
ax3.plot(win.time, -win.bx,'-r', color="xkcd:magenta",linewidth=1,label='Bx')
ax3.plot(win.time, -win.by, "g-", color="xkcd:orange", lw=1,label='By')
ax3.plot(win.time, win.bz, "b-", color="xkcd:azure", lw=1,label='Bz')


#plot vertical lines
ax3.plot_date([wicme_start_time,wicme_start_time],[-500,500],'-k',linewidth=1)            
ax3.plot_date([wmo_start_time,wmo_start_time],[-500,500],'-k',linewidth=1)            
ax3.plot_date([wmo_end_time,wmo_end_time],[-500,500],'-k',linewidth=1)            


ax3.set_ylabel('B [nT, HEE]')
ax3.set_xlim(plotstart,plotend)
ax3.set_ylim(-blim,blim)
ax3.legend(loc=1,ncol=1,fontsize=11)
ax3.xaxis.set_minor_locator(matplotlib.ticker.MultipleLocator(1/24*6))
ax3.grid(True)



#position on figure
#textstart=plotstart+datetime.timedelta(hours=2)
#plt.text(textstart,25,'HEEQ R/lat/lon: '+str(np.round(win.r[wicme_start_ind],3))+' ' +str(np.round(win.lat[wicme_start_ind],2))+'  '+\
#         str(np.round(win.lon[wicme_start_ind],2)),fontsize=13)


#plt.tight_layout()



############################## Wind SWE

ax4.plot(win.time, win.vt, color="xkcd:magenta",linewidth=1,label='Vp Wind')

ax4.plot_date([wicme_start_time,wicme_start_time],[-500,1000],'-k',linewidth=1)            
ax4.plot_date([wmo_start_time,wmo_start_time],[-500,1000],'-k',linewidth=1)            
ax4.plot_date([wmo_end_time,wmo_end_time],[-500,1000],'-k',linewidth=1)            

ax4.set_ylabel('V [km $\mathrm{s^{-1}}$]')
ax4.set_xlim(plotstart,plotend)
ax4.set_ylim(100,550)
ax4.grid(True)


#HUX
#ax4.plot_date(wsa_all_datetime, wsa_all_earth,'-k')

#THUX
ax4.plot_date(wsa_apr_thux_datetime, wsa_apr_thux_earth,linestyle='--',markersize=0,color="xkcd:magenta",label='V HUX Earth')
ax4.plot_date(wsa_apr_thux_datetime, wsa_apr_thux_solo,linestyle='--',markersize=0,color="xkcd:azure", label='V HUX Solar Orbiter')

ax4.xaxis.set_minor_locator(matplotlib.ticker.MultipleLocator(1/24*6))
#ax4.xaxis.grid(True, which='minor')

ax4.legend(loc=2,bbox_to_anchor=(0.11,0.99),fontsize=11)




#density
ax5=ax4.twinx()
ax5.plot(win.time, win.np, color="xkcd:orange", lw=1,label='Np Wind')
ax5.set_ylim(0,100)
ax5.set_ylabel('Np $[\mathrm{cm^{-3}}]$')


ax5.legend(loc=2,bbox_to_anchor=(0.0,0.4),fontsize=11)

ax5.xaxis.set_major_formatter( matplotlib.dates.DateFormatter('%b-%d') )




############################### other settings


#ax1.set_xticklabels([])
#ax2.set_xticklabels([])
#ax3.set_xticklabels([])
ax1.xaxis.set_major_formatter( matplotlib.dates.DateFormatter('%b-%d') )
ax2.xaxis.set_major_formatter( matplotlib.dates.DateFormatter('%b-%d') )
ax3.xaxis.set_major_formatter( matplotlib.dates.DateFormatter('%b-%d') )




plt.tight_layout()

#annotations

vshift=0.25
plt.annotate('(a)',[0.02,0.93],xycoords='figure fraction',fontsize=20)
plt.annotate('(b)',[0.49,0.98],xycoords='figure fraction',fontsize=20)
plt.annotate('(c)',[0.49,0.98-vshift*1],xycoords='figure fraction',fontsize=20)
plt.annotate('(d)',[0.49,0.98-vshift*2],xycoords='figure fraction',fontsize=20)
plt.annotate('(e)',[0.49,0.98-vshift*3],xycoords='figure fraction',fontsize=20)


ax1.text(0.02,0.86,'Solar Orbiter',transform=ax1.transAxes,fontsize=18,bbox=dict(facecolor='white', alpha=1))
ax2.text(0.02,0.86,'Bepi Colombo',transform=ax2.transAxes,fontsize=18, bbox=dict(facecolor='white', alpha=1))
ax3.text(0.02,0.86,'Wind',transform=ax3.transAxes,fontsize=18,bbox=dict(facecolor='white', alpha=1))
ax4.text(0.02,0.86,'Wind',transform=ax4.transAxes,fontsize=18,bbox=dict(facecolor='white', alpha=1))



#time
#ax5.text(1.4,1.3,mdates.num2date(frame_time_num).strftime("%Y %B %d  %H:%M")+' UT',fontsize=fsize+3, ha='left',c='black')
plt.annotate(mdates.num2date(frame_time_num).strftime("%Y %B %d  %H:%M")+' UT',[0.17,0.90],xycoords='figure fraction',fontsize=23)


#for ticks in polar plot may use
#https://stackoverrun.com/de/q/5522696


plt.savefig('/home/cmoestl/pycode/heliocats/results/davies_2020/figure1_overview5.png')
plt.savefig('/home/cmoestl/pycode/heliocats/results/davies_2020/figure1_overview5.pdf')

### 

### Shock analysis not shown in the paper

#### Shock analysis Wind

In [ ]:
%matplotlib inline

plt.figure(1,figsize=(20, 13),dpi=50)

#define subplot grid
ax3 = plt.subplot2grid((2, 1), (0, 0))  #
ax4 = plt.subplot2grid((2, 1), (1, 0))  #
#plt.title('Wind MFI')

ax3.plot(win.time, win.bt, "k-", lw=1,label='Btotal')
ax3.plot(win.time, -win.bx,'-r', color="xkcd:magenta",linewidth=1,label='Bx')
ax3.plot(win.time, -win.by, "g-", color="xkcd:orange", lw=1,label='By')
ax3.plot(win.time, win.bz, "b-", color="xkcd:azure", lw=1,label='Bz')


#plot vertical lines
ax3.plot_date([wicme_start_time,wicme_start_time],[-500,500],'-k',linewidth=1)            
ax3.plot_date([wmo_start_time,wmo_start_time],[-500,500],'-k',linewidth=1)            
ax3.plot_date([wmo_end_time,wmo_end_time],[-500,500],'-k',linewidth=1)            


ax3.set_ylabel('B [nT, HEE]')
ax3.set_xlim(mdates.date2num(wicme_start_time)-0.02,mdates.date2num(wicme_start_time)+0.02)
ax3.set_ylim(-blim,blim)
ax3.legend(loc=1,ncol=1,fontsize=11)
ax3.xaxis.set_minor_locator(matplotlib.ticker.MultipleLocator(1/24*6))
ax3.grid(True)




ax4.plot(win.time, win.vt, color="xkcd:magenta",linewidth=1,label='Vp Wind')


ax4.plot_date([wicme_start_time,wicme_start_time],[-500,1000],'-k',linewidth=1)            
ax4.plot_date([wmo_start_time,wmo_start_time],[-500,1000],'-k',linewidth=1)            
ax4.plot_date([wmo_end_time,wmo_end_time],[-500,1000],'-k',linewidth=1)            

ax4.set_ylabel('V [km $\mathrm{s^{-1}}$]')
ax4.set_xlim(mdates.date2num(wicme_start_time)-0.02,mdates.date2num(wicme_start_time)+0.02)
ax4.set_ylim(100,550)
ax4.grid(True)


#shock analysis

#normal

#sign bx by for GSE to HEE
#from 20 min to 10 minutes before and after averages
bu=np.array([np.mean(-win.bx[wicme_start_ind-10:wicme_start_ind-5]),np.mean(-win.by[wicme_start_ind-10:wicme_start_ind-5]),np.mean(win.bz[wicme_start_ind-10:wicme_start_ind-5]) ])
bd=np.array([np.mean(-win.bx[wicme_start_ind+5:wicme_start_ind+10]),np.mean(-win.by[wicme_start_ind+5:wicme_start_ind+10]),np.mean(win.bz[wicme_start_ind+5:wicme_start_ind+10]) ])


bm1=bu-bd
bc=np.cross(bu,bd)

#180 deg uncertain
nshock=-np.cross(bm1,bc)/np.linalg.norm(np.cross(bm1,bc))

print('Ns',nshock)

#=0 print(np.dot(nshock,bc))

angle_to_radial=np.arccos(np.dot(np.array([1,0,0]),nshock) )

print('angle to radial ',angle_to_radial*180/np.pi)                                                               


angle_to_x_in_xy_plane=np.arctan(nshock[1]/nshock[0])
print('angle to x ',angle_to_x_in_xy_plane*180/np.pi)                                                               



#alfven mach number
#----------------

#theta_bn angle between bu and nshock

theta_bn=np.arccos(np.dot(bu/np.linalg.norm(bu),nshock) )

print('theta_bn ',theta_bn*180/np.pi)                                                               
print('oblique')



#plt.plot(win.time[wicme_start_ind-10:wicme_start_ind+10], win.bx[wicme_start_ind-10:wicme_start_ind+10])
#plt.plot(win.time[wicme_start_ind-10:wicme_start_ind+10], win.by[wicme_start_ind-10:wicme_start_ind+10])
#plt.plot(win.time[wicme_start_ind-10:wicme_start_ind+10], win.bz[wicme_start_ind-10:wicme_start_ind+10])

#### shock analysis SolO

In [ ]:
plt.figure(1,figsize=(20, 13),dpi=50)

#define subplot grid
ax3 = plt.subplot2grid((1, 1), (0, 0))  #
#plt.title('smd MFI')

ax3.plot(sm.time, sm.bt, "k-", lw=1,label='Btotal')
ax3.plot(sm.time, sm.bx,'-r', color="xkcd:magenta",linewidth=1,label='Bx')
ax3.plot(sm.time, sm.by, "g-", color="xkcd:orange", lw=1,label='By')
ax3.plot(sm.time, sm.bz, "b-", color="xkcd:azure", lw=1,label='Bz')


#plot vertical lines
ax3.plot_date([sicme_start_time,sicme_start_time],[-500,500],'-k',linewidth=1)            
ax3.plot_date([smo_start_time,smo_start_time],[-500,500],'-k',linewidth=1)            
ax3.plot_date([smo_end_time,smo_end_time],[-500,500],'-k',linewidth=1)            


ax3.set_ylabel('B [nT, RTN]')
ax3.set_xlim(mdates.date2num(sicme_start_time)-0.02,mdates.date2num(sicme_start_time)+0.02)
ax3.set_ylim(-blim,blim)
ax3.legend(loc=1,ncol=1,fontsize=11)
ax3.xaxis.set_minor_locator(matplotlib.ticker.MultipleLocator(1/24*6))
ax3.grid(True)





#normal

#rtn
#from 20 min to 10 minutes before and after averages
bu=np.array([np.mean(sm.bx[sicme_start_ind-20:sicme_start_ind-10]),np.mean(sm.by[sicme_start_ind-20:sicme_start_ind-10]),np.mean(sm.bz[sicme_start_ind-20:sicme_start_ind-10]) ])
bd=np.array([np.mean(sm.bx[sicme_start_ind+10:sicme_start_ind+20]),np.mean(sm.by[sicme_start_ind+10:sicme_start_ind+20]),np.mean(sm.bz[sicme_start_ind+10:sicme_start_ind+20]) ])


bm1=bu-bd
bc=np.cross(bu,bd)

#180 deg uncertain
nshock=-np.cross(bm1,bc)/np.linalg.norm(np.cross(bm1,bc))

print('Ns',nshock)

#=0 print(np.dot(nshock,bc))

angle_to_radial=np.arccos(np.dot(np.array([1,0,0]),nshock) )

print('angle to radial ',angle_to_radial*180/np.pi)                                                               


angle_to_x_in_xy_plane=np.arctan(nshock[1]/nshock[0])
print('angle to x ',angle_to_x_in_xy_plane*180/np.pi)                                                               


#theta_bn angle between bu and nshock

theta_bn=np.arccos(np.dot(bu/np.linalg.norm(bu),nshock) )

print('theta_bn ',theta_bn*180/np.pi)                                                               
print('oblique')



#### is the shock driven ? 

#### shock analysis Bepi

In [ ]:
plt.figure(1,figsize=(20, 13),dpi=50)

#define subplot grid
ax3 = plt.subplot2grid((1, 1), (0, 0))  #
#plt.title('smd MFI')

ax3.plot(bm.time, bm.bt, "k-", lw=1,label='Btotal')
ax3.plot(bm.time, bm.bx,'-r', color="xkcd:magenta",linewidth=1,label='Bx')
ax3.plot(bm.time, bm.by, "g-", color="xkcd:orange", lw=1,label='By')
ax3.plot(bm.time, bm.bz, "b-", color="xkcd:azure", lw=1,label='Bz')


#plot vertical lines
ax3.plot_date([bicme_start_time,bicme_start_time],[-500,500],'-k',linewidth=1)            
ax3.plot_date([bmo_start_time,bmo_start_time],[-500,500],'-k',linewidth=1)            
ax3.plot_date([bmo_end_time,bmo_end_time],[-500,500],'-k',linewidth=1)            


ax3.set_ylabel('B [nT, HEE]')
ax3.set_xlim(mdates.date2num(bicme_start_time)-0.02,mdates.date2num(bicme_start_time)+0.02)
ax3.set_ylim(-blim,blim)
ax3.legend(loc=1,ncol=1,fontsize=11)
ax3.xaxis.set_minor_locator(matplotlib.ticker.MultipleLocator(1/24*6))
ax3.grid(True)





#normal

#rtn
#from 20 min to 10 minutes before and after averages
bu=np.array([np.mean(bm.bx[bicme_start_ind-10:bicme_start_ind-5]),np.mean(bm.by[bicme_start_ind-10:bicme_start_ind-5]),np.mean(bm.bz[bicme_start_ind-10:bicme_start_ind-5]) ])
bd=np.array([np.mean(bm.bx[bicme_start_ind+3:bicme_start_ind+8]),np.mean(bm.by[bicme_start_ind+3:bicme_start_ind+8]),np.mean(bm.bz[bicme_start_ind+3:bicme_start_ind+8]) ])


bm1=bu-bd
bc=np.cross(bu,bd)

#180 deg uncertain
nshock=-np.cross(bm1,bc)/np.linalg.norm(np.cross(bm1,bc))

print('Ns',nshock)

#=0 print(np.dot(nshock,bc))

angle_to_radial=np.arccos(np.dot(np.array([1,0,0]),nshock) )

print('angle to radial ',angle_to_radial*180/np.pi)                                                               


angle_to_x_in_xy_plane=np.arctan(nshock[1]/nshock[0])
print('angle to x ',angle_to_x_in_xy_plane*180/np.pi)                                                               


#theta_bn angle between bu and nshock

theta_bn=np.arccos(np.dot(bu/np.linalg.norm(bu),nshock) )

print('theta_bn ',theta_bn*180/np.pi)                                                               
print('parallel')



#### is the shock driven ? 

### positions directly from spice

In [ ]:
import datetime
import numpy as np
import spiceypy
import os

base = "/nas/helio/data/heliosat/kernels"
kernels = [
    "de430.bsp", "naif0012.tls", "heliospheric_v004u.tf", "pck00010.tpc", 
    "solo_ANC_soc-orbit_20200210-20301120_L004_V1_00062_V01.bsp", "solo_ANC_soc-sci-fk_V06.tf",
    "bc_mpo_fcp_00094_20181020_20251101_v01.bsp", "bc_sci_v06.tf"
]
for kernel in kernels:
    spiceypy.furnsh(os.path.join(base, kernel))
    
print("\nSOLAR SYSTEM BARYCENTER\n==============================")
t_solo = datetime.datetime(2020, 4, 19, 5, 6)
print(t_solo)
solo_pos = spiceypy.spkpos("SOLAR ORBITER", spiceypy.datetime2et(t_solo), "HEEQ", "NONE", "SOLAR_SYSTEM_BARYCENTER")[0]
print("solo lon: ", np.arctan2(solo_pos[1], solo_pos[0]) * 360 / 2 / np.pi)
print("solo lat: ", np.arctan2(solo_pos[2], solo_pos[0]) * 360 / 2 / np.pi)
t_bepi = datetime.datetime(2020, 4, 20, 3, 9)
bepi_pos = spiceypy.spkpos("BEPICOLOMBO MPO", spiceypy.datetime2et(t_bepi), "HEEQ", "NONE", "SOLAR_SYSTEM_BARYCENTER")[0]

print("bepi lon: ", np.arctan2(bepi_pos[1], bepi_pos[0]) * 360 / 2 / np.pi)
print("bepi lat: ", np.arctan2(bepi_pos[2], bepi_pos[0]) * 360 / 2 / np.pi)
print("\nSUN\n==============================")

t_solo = datetime.datetime(2020, 4, 19, 5, 6)
solo_pos = spiceypy.spkpos("SOLAR ORBITER", spiceypy.datetime2et(t_solo), "HEEQ", "NONE", "SUN")[0]
print("solo lon: ", np.arctan2(solo_pos[1], solo_pos[0]) * 360 / 2 / np.pi)
print("solo lat: ", np.arctan2(solo_pos[2], solo_pos[0]) * 360 / 2 / np.pi)

t_bepi = datetime.datetime(2020, 4, 20, 3, 9)
bepi_pos = spiceypy.spkpos("BEPICOLOMBO MPO", spiceypy.datetime2et(t_bepi), "HEEQ", "NONE", "SUN")[0]
print("bepi lon: ", np.arctan2(bepi_pos[1], bepi_pos[0]) * 360 / 2 / np.pi)
print("bepi lat: ", np.arctan2(bepi_pos[2], bepi_pos[0]) * 360 / 2 / np.pi)

SolO times: 2020-04-19 05:06:00 2020-04-19 08:59:00 2020-04-20 09:15:00

HEEQ position of SolO r/lat/lon at icme_start_time: 0.806 -3.9 -4.61

In [ ]:
t_snap_solo = datetime.datetime(2020, 4, 19, 5, 6)
solo_pos = heliosat.SOLO().trajectory(t_snap_solo, frame="HEEQ")
print("solo lon: ", np.arctan2(solo_pos[1], solo_pos[0]) * 360 / 2 / np.pi)
print("solo lat: ", np.arctan2(solo_pos[2], solo_pos[0]) * 360 / 2 / np.pi)

t_snap_wind = datetime.datetime(2020, 4, 20, 1, 34)
wind_pos = heliosat.WIND().trajectory(t_snap_wind, frame="HEEQ")
print("wind lon: ", np.arctan2(wind_pos[1], wind_pos[0]) * 360 / 2 / np.pi)
print("wind lat: ", np.arctan2(wind_pos[2], wind_pos[0]) * 360 / 2 / np.pi)

t_snap_bepi = datetime.datetime(2020, 4, 20, 3, 9)
bepi_pos = heliosat.BEPI().trajectory(t_snap_bepi, frame="HEEQ")
print("bepi lon: ", np.arctan2(bepi_pos[1], bepi_pos[0]) * 360 / 2 / np.pi)
print("bepi lat: ", np.arctan2(bepi_pos[2], bepi_pos[0]) * 360 / 2 / np.pi)